In [1]:
import os, sys
sys.path.append(os.getcwd()+"/../../..")
sys.path.append(os.getcwd()+"/../..")
sys.path.append(os.getcwd()+"/..")
from noise import *
from qstates import *
from utils import *
from quantitative.graph import Graph
from quantitative.game import *
from simulator import *
from math import isclose
from settings import ibm_noise_model, backend
from copy import deepcopy
Precision.PRECISION = 3

In [2]:
initial_qs = QuantumState(0)
initial_qs
initial_cs = ClassicalState()
def w(a):
    return 1

In [3]:
def get_bell_state(i, with_noise=True, seed=None):
    assert i >= 0 and i < 4
    if not with_noise:
        qpu = QSimulator(NoiseModel(), 1)
    else:
        assert seed is not None
        qpu = QSimulator(noise_model, seed)
    qpu.apply_instruction(Instruction.H, m(0))
    qpu.apply_instruction(Instruction.CNOT, m(1), [m(0)])
    if i == 0:
        pass
    elif i == 1:
        qpu.apply_instruction(Instruction.X, m(0))
    elif i == 2:
        qpu.apply_instruction(Instruction.Z, m(0))
    else:
        assert i == 3
        qpu.apply_instruction(Instruction.X, m(0))
        qpu.apply_instruction(Instruction.Z, m(0))
        
    return qpu.qmemory

In [4]:
def m (address):
    if address is None:
        return None
    if backend == FAKE_TENERIFE:            
        if address == 2:
            return 0
        if address == 0:
            return 2
        if address == 1:
            return 1
    if backend == FAKE_JAKARTA:
        if address == 0:
            return 5
        if address == 1:
            return 3
        if address == 2:
            return 1
    if backend == FAKE_JOHANNESBURG:
        if address == 0:
            return 14
        if address == 1:
            return 9
        if address == 2:
            return 13
        
    if backend == FAKE_LAGOS:
        if address == 0:
            return 4
        if address == 1:
            return 5
        if address == 2:
            return 6
    assert False
    
def inv_m (address):
    if address is None:
        return None
    if backend == FAKE_TENERIFE:            
        if address == 0:
            return 2
        if address == 2:
            return 0
        if address == 1:
            return 1
    if backend == FAKE_JAKARTA:
        if address == 5:
            return 0
        if address == 3:
            return 1
        if address == 1:
            return 2
    if backend == FAKE_JOHANNESBURG:
        if address == 14:
            return 0
        if address == 9:
            return 1
        if address == 13:
            return 2
        
    if backend == FAKE_LAGOS:
        if address == 4:
            return 0
        if address == 5:
            return 1
        if address == 6:
            return 2
    assert False

In [5]:
def is_target_qs(qs, address_space):
    prob00 = 0.0
    prob11 = 0.0
    for (basis, value) in qs.sparse_vector.items():
        qubit0 = (basis >> address_space(0)) & 1
        qubit1 = (basis >> address_space(1)) & 1
        if qubit0 != qubit1:
            probv = value * conjugate(value)
            if isinstance(probv, complex):
                probv = probv.real
            if not isclose(probv, 0.0, abs_tol=Precision.isclose_abstol): 
                return False
        else:
            if qubit0 == 0:
                prob00 += value * conjugate(value)
            else:
                prob11 += value * conjugate(value)

    if isinstance(prob00, complex):
        prob00 = prob00.real

    if isinstance(prob11, complex):
        prob11 = prob11.real
    if isclose(prob00, 0.0, abs_tol=Precision.isclose_abstol):
        return False
    
    return isclose(prob00, prob11) 

def is_not_answer(qs, address_space):
    return is_flip_bell_state(qs, address_space) or is_flip_bell_state_minus(qs, address_space)

In [6]:
def traditional_parity_check(noise_model, seed, qs):
    qpu = QSimulator(noise_model, seed)
    qpu.qmemory = qs
    qpu.apply_instruction(Instruction.CNOT, m(2), [m(0)])
    qpu.apply_instruction(Instruction.CNOT, m(2), [m(1)])
    outcome = qpu.apply_instruction(Instruction.MEAS, m(2))
    if outcome == 1:
        qpu.apply_instruction(Instruction.X, m(0))
    return qpu.qmemory, outcome, qpu.count_executed_instructions

def get_traditional_algo_stats(shots, bell_index =-1, bell_with_noise=False):
    total_correct = 0
    i = 0
    overall_correct = 0
    c = 0 # avg. executed instructions
    while shots > 0:
        if bell_index == -1:
            curr_state = get_bell_state(random.randint(0,3), with_noise=bell_with_noise, seed=i)
        else:
            curr_state = get_bell_state(bell_index)
        qs, outcome, count_ins = traditional_parity_check(noise_model, i, curr_state)
        c += count_ins
        is_bell = is_target_qs(qs, m)
        if is_bell:
            overall_correct += 1
        i+=1
        shots-=1
    assert shots == 0
    return round(overall_correct/i, 3), round(c/i, 3)

In [7]:
def execute_my_algo(shots, new_algo, initial_belief, bell_with_noise=False, with_robust_read=False, with_readout_err=True):    
    totalc = 0
    correct_is_correct = 0
    total_errors = 0
    error_is_error= 0
    i = 0
    trials = 5
    c = 0
    while shots > 0:
        state = get_bell_state(random.randint(0,3), with_noise=bell_with_noise, seed=i)
        qpu = QSimulator(noise_model, seed=i, with_readout_err=with_readout_err)
        qpu.qmemory = state
        r = new_algo(noise_model, w, seed=i, qpu=qpu,init_belief=initial_belief)
        outcome = r.error
        is_bell = is_target_qs(qpu.qmemory, m)
        c += qpu.count_executed_instructions

        if outcome == 1:
            trials-=1
            total_errors += 1
            if not is_bell:
                error_is_error += 1
        else:
            assert outcome == 0
            trials = 5
            shots -= 1
            totalc += 1
            if is_bell:
                correct_is_correct += 1
        i+=1
        if trials == 0:
            break
        
    assert i == totalc + total_errors
    assert shots == 0
    print("total executions: ", totalc+total_errors)
    if total_errors > 0:
        print("error detection acc.:", round(error_is_error/total_errors, 3))
    if totalc > 0:
        print("positives acc.:", round(correct_is_correct/totalc, 3))
    if total_errors + totalc > 0:
        print("total acc.", round((correct_is_correct+error_is_error)/(totalc + total_errors),3))
    print("avg. executed instructions: ", round(c/i, 3))
    


In [12]:
def execute_enumerated_algos(algorithm, shots= 1000, with_readout_err=True, bell_with_noise=False, is_bellman=False):
    totalc = 0
    correct_is_correct = 0
    total_errors = 0
    error_is_error= 0
    i = 0
    trials = 5
    c = 0
    while shots > 0:
        state = get_bell_state(random.randint(0,3), with_noise=bell_with_noise, seed=i)
        qpu = QSimulator(noise_model, seed=i, with_readout_err=with_readout_err)
        qpu.qmemory = state
        if not is_bellman:
            outcome, qpu = algorithm(qpu=qpu)
        else:
            result = algorithm(noise_model, seed=i, qpu=qpu, address_space=default_mapping)
            qpu.qmemory = result.qmemory
            if result.error:
                outcome = 1
            else:
                outcome = 0
        is_bell = is_target_qs(qpu.qmemory, m)
        c += qpu.count_executed_instructions

        if outcome == 1:
            trials-=1
            total_errors += 1
            if not is_bell:
                error_is_error += 1
        else:
            assert outcome == 0
            trials = 5
            shots -= 1
            totalc += 1
            if is_bell:
                correct_is_correct += 1
        i+=1
        if trials == 0:
            break
        
    assert i == totalc + total_errors
    assert shots == 0
    print("total executions: ", totalc+total_errors)
    if total_errors > 0:
        print("error detection acc.:", round(error_is_error/total_errors, 3))
    if totalc > 0:
        print("positives acc.:", round(correct_is_correct/totalc, 3))
    if total_errors + totalc > 0:
        print("total acc.", round((correct_is_correct+error_is_error)/(totalc + total_errors),3))
    print("avg. executed instructions: ", round(c/i, 3))

In [13]:
from tenerife_algos import *
backend = FAKE_TENERIFE
noise_model = get_ibm_noise_model(backend)
execute_enumerated_algos(simulate_algo, shots=1000, is_bellman=True)

Adding image coupler for 2-1 qubits
Adding image coupler for 2-3 qubits
Adding image coupler for 2-0 qubits
Adding image coupler for 0-1 qubits
Adding image coupler for 4-2 qubits
Adding image coupler for 1-4 qubits
Adding image coupler for 4-1 qubits
total executions:  1000
positives acc.: 0.948
total acc. 0.948
avg. executed instructions:  4.887


# Fake Tenerife

In [9]:
from bell_tenerife import *

ModuleNotFoundError: No module named 'bell_tenerife'

In [14]:
backend = FAKE_TENERIFE
noise_model = get_ibm_noise_model(backend)

Adding image coupler for 2-1 qubits
Adding image coupler for 2-3 qubits
Adding image coupler for 2-0 qubits
Adding image coupler for 0-1 qubits
Adding image coupler for 4-2 qubits
Adding image coupler for 1-4 qubits
Adding image coupler for 4-1 qubits


In [11]:
get_traditional_algo_stats(8000)

(0.892, 3.501)

In [11]:
Precision.PRECISION = 3
execute_my_algo(8000, simulate_bell_tenerife, None, with_robust_read=False)

total executions:  8266
error detection acc.: 0.556
positives acc.: 0.964
total acc. 0.951
avg. executed instructions:  5.706


In [11]:
from tenerife_algos import *
# for i in range(len(tenerife_algorithms)):
#     print(i)
#     execute_enumerated_algos(tenerife_algorithms[i], shots=100)
#     print("")

# Fake Jakarta

In [9]:
noise_model = get_ibm_noise_model(FAKE_JAKARTA)

Adding coupler for 5<->1 qubits


In [10]:
get_traditional_algo_stats(8000)

(0.961, 3.5)

In [10]:
from bell_jakarta import *

In [12]:
execute_my_algo(8000, simulate_bell_jakarta, None)

total executions:  8222
error detection acc.: 0.986
positives acc.: 0.978
total acc. 0.978
avg. executed instructions:  4.509


In [13]:
from jakarta_algos import *
for i in range(len(jakarta_algorithms)):
    print(i)
    execute_enumerated_algos(jakarta_algorithms[i], shots=8000)
    print("")

0
total executions:  8223
error detection acc.: 0.996
positives acc.: 0.979
total acc. 0.979
avg. executed instructions:  4.504



# Fake Johannesburg

In [16]:
backend = FAKE_JOHANNESBURG
noise_model = get_ibm_noise_model(backend)

Adding coupler for 13-9 qubits
Adding coupler for 13-9 qubits


In [10]:
get_traditional_algo_stats(8000)

(0.867, 3.5)

In [10]:
from bell_johann import *

In [11]:
execute_my_algo(8000, simulate_bell_johann, None)

total executions:  8000
positives acc.: 0.939
total acc. 0.939
avg. executed instructions:  4.474


In [12]:
from johann_algos import *
for i in range(len(johannesburg_algorithms)):
    print(i)
    execute_enumerated_algos(johannesburg_algorithms[i], shots=8000)
    print("")

0
total executions:  8000
positives acc.: 0.936
total acc. 0.936
avg. executed instructions:  4.475

1
total executions:  8000
positives acc.: 0.929
total acc. 0.929
avg. executed instructions:  4.479

2
total executions:  8000
positives acc.: 0.935
total acc. 0.935
avg. executed instructions:  4.475



In [13]:
from qiskit.providers.fake_provider import FakeJohannesburg
import numpy as np
from qiskit_aer import AerSimulator
ibm_noise_model = IBMNoiseModel.from_backend(FakeJohannesburg())
d = ibm_noise_model.to_dict()
c14_9 = None
c9_14 = None
for e in d['errors']:
    for q in e['gate_qubits']:
        if (14, 9)  == q:
            assert c14_9 is None
            c14_9 = deepcopy(e)
        elif (9, 14)in e['gate_qubits']:
            assert c9_14 is None
            c9_14 = deepcopy(e)
c14_9['gate_qubits'] = [(13, 9)]
c9_14['gate_qubits'] = [(9, 13)]
d['errors'].append(c14_9)
d['errors'].append(c9_14)
ibm_noise_model = IBMNoiseModel.from_dict(d)

from qiskit.extensions import HGate, XGate, ZGate, CXGate
noisefree_h = HGate(label="H1")
noisefree_x = XGate(label="X1")
noisefree_z = ZGate(label="Z1")
noisefree_cx = CXGate(label="CX1")

ibm_noise_model.add_basis_gates(['H1', "X1", "Z1", "CX1"])

/var/folders/jb/g9mmpv8n62190fgx4t722rnr0000gn/T/ipykernel_9879/4227381476.py:20: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  ibm_noise_model = IBMNoiseModel.from_dict(d)


In [14]:
def get_qiskit_bell_state(i):
    assert i >= 0 and i < 4
    qr = QuantumRegister(3)
    cr = ClassicalRegister(3)
    qc = QuantumCircuit(qr, cr)
    
    qc.append(noisefree_h, [0]) 
    qc.append(noisefree_cx, [0, 1]) 
    
    if i == 0:
        pass
    elif i == 1:
        qc.append(noisefree_x, [0]) 
    elif i == 2:
        qc.append(noisefree_z, [0]) 
    else:
        assert i == 3
        qc.append(noisefree_x, [0]) 
        qc.append(noisefree_z, [0]) 
    return qc, qr, cr

def get_johann_algo2(i):
    qc, qr, cr = get_qiskit_bell_state(i)
    qc.cx(qr[1], qr[2])
    qc.h(qr[0])
    qc.cx(qr[0], qr[1])
    qc.measure(qr[2], cr[2])

    with qc.if_test((cr[2], True)) as else_:
        qc.x(qr[0])
    with else_:
        pass
    qc.save_statevector('psi2', pershot=True)
    return qc
    
def default_ibm_algo(i):
    qc, qr, cr = get_qiskit_bell_state(i)
    qc.cx(qr[1], qr[2])
    qc.cx(qr[0], qr[2])
    qc.measure(qr[2], cr[2])

    with qc.if_test((cr[2], True)) as else_:
        qc.x(qr[0])
    with else_:
        pass
    qc.save_statevector('psi2', pershot=True)
    return qc

In [15]:
def ibm_is_target_qs(statev) -> bool:
    prob00 = 0.0
    prob11 = 0.0
    for basis in range(8):
        value = statev[basis]
        b_str = int_to_bin(basis)
        while len(b_str) < 3:
            b_str += "0"
        qubit0 = (b_str[0] == '1')
        qubit1 = (b_str[2] == '1')
        if qubit0 != qubit1:
            probv = value * conjugate(value)
            if isinstance(probv, complex):
                probv = probv.real
            if not isclose(probv, 0.0, abs_tol=Precision.isclose_abstol): 
                return False
        else:
            if qubit0 == 0:
                prob00 += value*conjugate(value)
            else:
                prob11 += value*conjugate(value)

    if isinstance(prob00, complex):
        prob00 = simplify(prob00.real)
        
    if isinstance(prob11, complex):
        prob11 = simplify(prob11.real)
        
    if isclose(prob00, 0.0, abs_tol=Precision.isclose_abstol):
        return False
    
    return isclose(prob00, prob11)



In [18]:
sim_noise = AerSimulator(method ='statevector', noise_model=ibm_noise_model)

shots = 2000

acc = 0

for b in range(4):
    qc = get_johann_algo2(b)
    circ_tnoise = transpile(qc, sim_noise, optimization_level=1, initial_layout=[14, 9, 13])
    # Run and get counts
    result = sim_noise.run(circ_tnoise, shots=shots).result()
    statevs = np.asarray(result.data()['psi2'])
    for shot in range(shots):
        statev = np.asarray(statevs[shot])
        if ibm_is_target_qs(statev):
            acc += 1
                
            
    
print(round(acc/(4*shots), 3))

0.913


In [104]:
round(0.936-0.918,3)

0.018

In [102]:
sim_noise = AerSimulator(method ='statevector', noise_model=ibm_noise_model)

shots = 2000

acc = 0

for b in range(4):
    qc = default_ibm_algo(b)
    circ_tnoise = transpile(qc, sim_noise, optimization_level=1, initial_layout=[14, 9, 13])
    # Run and get counts
    result = sim_noise.run(circ_tnoise, shots=shots).result()
    statevs = np.asarray(result.data()['psi2'])
    for shot in range(shots):

        statev = np.asarray(statevs[shot])
        
        if ibm_is_target_qs(statev):
            acc += 1
print(round(acc/(4*shots), 3))

0.879


In [105]:
round(0.867 - 0.879,3)

-0.012

# Fake Lagos

In [9]:
backend = FAKE_LAGOS
noise_model = get_ibm_noise_model(backend)

Adding coupler for 4<->6 qubits


In [26]:
get_traditional_algo_stats(8000)

(0.981, 3.489)

In [10]:
from bell_lagos import *

In [11]:
Precision.PRECISION = 4

In [12]:
execute_my_algo(8000, simulate_bell_lagos, None)

total executions:  8127
error detection acc.: 0.528
positives acc.: 0.988
total acc. 0.98
avg. executed instructions:  4.494


In [13]:
from lagos_algos import *
for i in range(len(lagos_algorithms)):
    print(i)
    execute_enumerated_algos(lagos_algorithms[i], shots=8000)
    print("")

0
total executions:  8108
error detection acc.: 0.565
positives acc.: 0.985
total acc. 0.98
avg. executed instructions:  4.485

1
total executions:  8118
error detection acc.: 0.5
positives acc.: 0.986
total acc. 0.978
avg. executed instructions:  4.492



# Brute Force Algorithms

In [7]:
backend = FAKE_JOHANNESBURG
noise_model = get_ibm_noise_model(backend)

In [9]:
channels = [
    ([Instruction.CNOT], [2], [0], None),
    ([Instruction.CNOT], [2], [1], None),
    ([Instruction.CNOT], [1], [0], None),
    ([Instruction.MEAS], [2], [None], None),
    ([Instruction.X], [0], [None], None),
    ([Instruction.H], [0], [None], None),
]